In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_pickle('features.csv')
df['mean_features'] = df['features'].apply(lambda x: np.mean(x, axis=1))

In [3]:
df['gender'] = df['gender'].replace(['male', 'male ', 'مرد'], 1).replace(['female', 'Female'], 0)
X = np.array(df.iloc[:, -1].tolist())
y = np.array(df['gender'])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1000, test_size=0.25, stratify=y)
# X_train = StandardScaler().fit(X_train).transform(X_train)
# X_test = StandardScaler().fit(X_train).transform(X_test)



/tmp/ipykernel_43939/112978659.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['gender'] = df['gender'].replace(['male', 'male ', 'مرد'], 1).replace(['female', 'Female'], 0)


In [4]:
print("Label distribution in the whole dataset:")
print(np.bincount(y) / len(y))

print("Label distribution in the training set:")
print(np.bincount(y_train) / len(y_train))

print("Label distribution in the test set:")
print(np.bincount(y_test) / len(y_test))

Label distribution in the whole dataset:
[0.21880172 0.78119828]
Label distribution in the training set:
[0.21871552 0.78128448]
Label distribution in the test set:
[0.21906023 0.78093977]


In [5]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [6]:
# clf = GaussianNB().fit(X_train, y_train)

# p_test = clf.predict(X_test)

# print(classification_report(y_test, p_test))

In [7]:
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, classification_report, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate(clf, report=True):
    print(f'---------{type(clf).__name__}---------')
    start_time = time.time()
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f'F1-score = {round(f1, 2)}')
    
    if report:
        print(classification_report(y_test, y_pred))

        conf_mat = confusion_matrix(y_test, y_pred)
        sns.heatmap(conf_mat, annot=True, fmt='d')
        plt.title('Confusion Matrix')
        plt.show()

        y_pred_prob = clf.predict_proba(X_test)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr, tpr)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve (AUC={roc_auc_score(y_test, y_pred_prob)})')
        plt.show()
    
    end_time = time.time()
    print(f"-- Took: {round(end_time - start_time, 3)} seconds")
    
    return f1


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import accuracy_score

# Assuming X_train, y_train, X_test, y_test are your data
classifiers = [
    GaussianNB(), 
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    MLPClassifier(),
]


scores = [evaluate(clf, False) for clf in classifiers]


---------GaussianNB---------
F1-score = 0.68
-- Took: 0.007 seconds
---------SVC---------
F1-score = 0.73
-- Took: 9.649 seconds
---------DecisionTreeClassifier---------
F1-score = 0.92
-- Took: 0.12 seconds
---------RandomForestClassifier---------
F1-score = 0.97
-- Took: 1.759 seconds
---------AdaBoostClassifier---------


/home/erfan/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


F1-score = 0.86
-- Took: 0.765 seconds
---------GradientBoostingClassifier---------
F1-score = 0.92
-- Took: 3.457 seconds
---------MLPClassifier---------
F1-score = 0.92
-- Took: 14.375 seconds
